# Amazon SageMaker BATCH Transform examples
With [Amazon SageMaker Batch Transform](https://docs.aws.amazon.com/sagemaker/latest/dg/batch-transform.html), Use batch transform when you need to do the following:

    Preprocess datasets to remove noise or bias that interferes with training or inference from your dataset.

    Get inferences from large datasets.

    Run inference when you don't need a persistent endpoint.

    Associate input records with inferences to assist the interpretation of results.

To filter input data before performing inferences or to associate input records with inferences about those records, see Associate Prediction Results with Input Records. For example, you can filter input data to provide context for creating and interpreting reports about the output data.For applications which require consistently low inference latency, a traditional endpoint is still the best choice.

To split input files into mini-batches when you create a batch transform job, set the SplitType parameter value to Line. If SplitType is set to None or if an input file can't be split into mini-batches, SageMaker uses the entire input file in a single request. Note that Batch Transform doesn't support CSV-formatted input that contains embedded newline characters. You can control the size of the mini-batches by using the BatchStrategy and MaxPayloadInMB parameters. MaxPayloadInMB must not be greater than 100 MB. If you specify the optional MaxConcurrentTransforms parameter, then the value of (MaxConcurrentTransforms * MaxPayloadInMB) must also not exceed 100 MB.

If the batch transform job successfully processes all of the records in an input file, it creates an output file with the same name and the .out file extension. For multiple input files, such as input1.csv and input2.csv, the output files are named input1.csv.out and input2.csv.out. The batch transform job stores the output files in the specified location in Amazon S3, such as s3://awsexamplebucket/output/.



![](./cw_charts/BatchTransform.png)





### Contents

1. [Generate synthetic data for housing models](#Generate-synthetic-data-for-housing-models)
1. [TRANSFORM the raw housing data using Scikit Learn model](#Preprocess-synthetic-housing-data-using-scikit-learn)
1. [Clean up](#CleanUp)


## Section 1 - Generate synthetic data for housing models <a id='Generate-synthetic-data-for-housing-models'></a>

In this section, you will generate synthetic data that will be used to train the linear learner models.  The data generated consists of 6 numerical features - the year the house was built in, house size in square feet, number of bedrooms, number of bathroom, the lot size and number of garages and two categorial features - deck and front_porch.  

In [17]:
import numpy as np
import pandas as pd
import json
import datetime
import time
import boto3
import sagemaker
import os

from time import gmtime, strftime
from random import choice

from sagemaker import get_execution_role

from sagemaker.multidatamodel import MULTI_MODEL_CONTAINER_MODE
from sagemaker.multidatamodel import MultiDataModel

from sklearn.model_selection import train_test_split

In [18]:
NUM_HOUSES_PER_LOCATION = 1000
LOCATIONS  = ['NewYork_NY',    'LosAngeles_CA',   'Chicago_IL',    'Houston_TX',   'Dallas_TX',
              'Phoenix_AZ',    'Philadelphia_PA', 'SanAntonio_TX', 'SanDiego_CA',  'SanFrancisco_CA']
MAX_YEAR = 2019

In [19]:
def gen_price(house):
    """Generate price based on features of the house"""
    
    if house['FRONT_PORCH'] == 'y':
        garage = 1
    else:
        garage = 0
        
    if house['FRONT_PORCH'] == 'y':
        front_porch = 1
    else:
        front_porch = 0
        
    price = int(150 * house['SQUARE_FEET'] + \
                10000 * house['NUM_BEDROOMS'] + \
                15000 * house['NUM_BATHROOMS'] + \
                15000 * house['LOT_ACRES'] + \
                10000 * garage + \
                10000 * front_porch + \
                15000 * house['GARAGE_SPACES'] - \
                5000 * (MAX_YEAR - house['YEAR_BUILT']))
    return price

In [20]:
def gen_yes_no():
    """Generate values (y/n) for categorical features"""
    answer = choice(['y', 'n'])
    return answer

In [21]:
def gen_random_house():
    """Generate a row of data (single house information)"""
    house = {'SQUARE_FEET':    np.random.normal(3000, 750),
             'NUM_BEDROOMS':  np.random.randint(2, 7),
             'NUM_BATHROOMS': np.random.randint(2, 7) / 2,
             'LOT_ACRES':     round(np.random.normal(1.0, 0.25), 2),
             'GARAGE_SPACES': np.random.randint(0, 4),
             'YEAR_BUILT':    min(MAX_YEAR, int(np.random.normal(1995, 10))),
             'FRONT_PORCH':   gen_yes_no(),
             'DECK':          gen_yes_no()
            }
    
    price = gen_price(house)
    
    return [house['YEAR_BUILT'],   
            house['SQUARE_FEET'], 
            house['NUM_BEDROOMS'], 
            house['NUM_BATHROOMS'], 
            house['LOT_ACRES'],    
            house['GARAGE_SPACES'],
            house['FRONT_PORCH'],    
            house['DECK'], 
            price]

In [22]:
def gen_houses(num_houses):
    """Generate housing dataset"""
    house_list = []
    
    for _ in range(num_houses):
        house_list.append(gen_random_house())
        
    df = pd.DataFrame(
        house_list, 
        columns=[
            'YEAR_BUILT',    
            'SQUARE_FEET',  
            'NUM_BEDROOMS',            
            'NUM_BATHROOMS',
            'LOT_ACRES',
            'GARAGE_SPACES',
            'FRONT_PORCH',
            'DECK', 
            'PRICE']
    )
    return df

In [23]:
def save_data_locally(location, train, test): 
    """Save the housing data locally"""
    os.makedirs('data/{0}/train'.format(location), exist_ok=True)
    train.to_csv('data/{0}/train/train.csv'.format(location), sep=',', header=False, index=False)
       
    os.makedirs('data/{0}/test'.format(location), exist_ok=True)
    test.to_csv('data/{0}/test/test.csv'.format(location), sep=',', header=False, index=False) 

In [29]:
#Generate housing data for multiple locations.
#Change "PARALLEL_TRAINING_JOBS " to a lower number to limit the number of training jobs and models. Or to a higher value to experiment with more models.

PARALLEL_TRAINING_JOBS = 1

for loc in LOCATIONS[:PARALLEL_TRAINING_JOBS]:
    houses = gen_houses(NUM_HOUSES_PER_LOCATION)
    
    #Spliting data into train and test in 90:10 ratio
    #Not splitting the train data into train and val because its not preprocessed yet
    train, test = train_test_split(houses, test_size=0.1)
    save_data_locally(loc, train, test)


In [30]:
#Shows the first few lines of data.
houses.head()

,YEAR_BUILT,SQUARE_FEET,NUM_BEDROOMS,NUM_BATHROOMS,LOT_ACRES,GARAGE_SPACES,FRONT_PORCH,DECK,PRICE
0,1996,1843.122187,4,3.0,1.03,0,n,n,261918
1,1991,2929.421598,4,2.5,0.80,0,n,n,388913
2,1982,3086.040886,5,2.5,0.94,1,y,n,414506
3,1984,2087.861573,6,1.5,0.86,0,y,n,253579
4,2000,2213.243535,5,2.5,1.49,2,n,n,376836


## Section 2 - Preprocess the raw housing data using Scikit Learn <a id='Preprocess-synthetic-housing-data-using-scikit-learn'></a>

In this section, the categorical features of the data (deck and porch) are pre-processed using sklearn to convert them to one hot encoding representation.  

#### We launch 4 PARALLEL jobs and hence we seemingly create 4 Transformers, but in reality they are just the SAME estimator being run again

In [31]:
%%writefile scripts/sklearn_preprocessor_batch.py
from __future__ import print_function

import argparse
import csv
import json
import os
import shutil
import sys
import time
from io import StringIO

import numpy as np
import pandas as pd
from sagemaker_containers.beta.framework import (
    content_types,
    encoders,
    env,
    modules,
    transformer,
    worker,
)
from sklearn.compose import ColumnTransformer
from sklearn.externals import joblib
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import Binarizer, OneHotEncoder, StandardScaler

# Since we get a headerless CSV file we specify the column names here.
feature_columns_names = [
    "YEAR_BUILT",
    "SQUARE_FEET",
    "NUM_BEDROOMS",
    "NUM_BATHROOMS",
    "LOT_ACRES",
    "GARAGE_SPACES",
    "FRONT_PORCH",
    "DECK",
]

label_column = "PRICE"

feature_columns_dtype = {
    "YEAR_BUILT": str,
    "SQUARE_FEET": np.float64,
    "NUM_BEDROOMS": np.float64,
    "NUM_BATHROOMS": np.float64,
    "LOT_ACRES": np.float64,
    "GARAGE_SPACES": np.float64,
    "FRONT_PORCH": str,
    "DECK": str,
}

label_column_dtype = {"PRICE": np.float64}


if __name__ == "__main__":

    parser = argparse.ArgumentParser()

    # Sagemaker specific arguments. Defaults are set in the environment variables.
    parser.add_argument("--output-data-dir", type=str, default=os.environ["SM_OUTPUT_DATA_DIR"])
    parser.add_argument("--model-dir", type=str, default=os.environ["SM_MODEL_DIR"])
    parser.add_argument("--train", type=str, default=os.environ["SM_CHANNEL_TRAIN"])

    args = parser.parse_args()

    # Take the set of files and read them all into a single pandas dataframe
    input_files = [os.path.join(args.train, file) for file in os.listdir(args.train)]
    if len(input_files) == 0:
        raise ValueError(
            (
                "There are no files in {}.\n"
                + "This usually indicates that the train channel was incorrectly specified,\n"
                + "the data specification in S3 was incorrectly specified or the role specified\n"
                + "does not have permission to access the data.".format(args.train)
            )
        )

    for file in input_files:
        print("file :", file)

    raw_data = [pd.read_csv(file, header=None, names=feature_columns_names + [label_column])]

    concat_data = pd.concat(raw_data)

    print(concat_data)

    # This section is adapted from the scikit-learn example of using preprocessing pipelines:
    #
    # https://scikit-learn.org/stable/auto_examples/compose/plot_column_transformer_mixed_types.html
    #

    numeric_features = list(feature_columns_names)
    numeric_features.remove("FRONT_PORCH")
    numeric_features.remove("DECK")
    numeric_transformer = Pipeline(steps=[("scaler", StandardScaler())])

    categorical_features = ["FRONT_PORCH", "DECK"]
    categorical_transformer = Pipeline(steps=[("onehot", OneHotEncoder(handle_unknown="ignore"))])

    preprocessor = ColumnTransformer(
        transformers=[
            ("num", numeric_transformer, numeric_features),
            ("cat", categorical_transformer, categorical_features),
        ],
        remainder="drop",
    )

    preprocessor.fit(concat_data)

    joblib.dump(preprocessor, os.path.join(args.model_dir, "model.joblib"))

    print("saved model!")


def input_fn(input_data, content_type):
    """Parse input data payload

    We currently only take csv input. Since we need to process both labelled
    and unlabelled data we first determine whether the label column is present
    by looking at how many columns were provided.
    """
    if content_type == "text/csv":
        # Read the raw input data as CSV.
        df = pd.read_csv(StringIO(input_data), header=None)

        if len(df.columns) == len(feature_columns_names) + 1:
            # This is a labelled example, includes the ring label
            df.columns = feature_columns_names + [label_column]
        elif len(df.columns) == len(feature_columns_names):
            # This is an unlabelled example.
            df.columns = feature_columns_names

        return df
    else:
        raise ValueError("{} not supported by script!".format(content_type))


def output_fn(prediction, accept):
    """Format prediction output

    The default accept/content-type between containers for serial inference is JSON.
    We also want to set the ContentType or mimetype as the same value as accept so the next
    container can read the response payload correctly.
    """
    if accept == "application/json":
        instances = []
        for row in prediction.tolist():
            instances.append({"features": row})

        json_output = {"instances": instances}

        return worker.Response(json.dumps(json_output), mimetype=accept)
    elif accept == "text/csv":
        return worker.Response(encoders.encode(prediction, accept), mimetype=accept)
    else:
        raise RuntimeException("{} accept type is not supported by this script.".format(accept))


def predict_fn(input_data, model):
    """Preprocess input data

    We implement this because the default uses .predict(), but our model is a preprocessor
    so we want to use .transform().

    The output is returned in the following order:

        rest of features either one hot encoded or standardized
    """

    print("Input data type ", type(input_data))

    print(input_data)

    features = model.transform(input_data)

    print("features type ", type(features))

    print(features)

    features_array = features

    print("features_array ", type(features_array))

    print(features_array)

    if label_column in input_data:
        # Return the label (as the first column) and the set of features.
        return np.insert(features_array, 0, input_data[label_column], axis=1)
    else:
        # Return only the set of features
        return features


def model_fn(model_dir):
    """Deserialize fitted model"""
    preprocessor = joblib.load(os.path.join(model_dir, "model.joblib"))
    return preprocessor



Overwriting scripts/sklearn_preprocessor_batch.py


In [32]:
sm_client = boto3.client(service_name='sagemaker')
runtime_sm_client = boto3.client(service_name='sagemaker-runtime')
sagemaker_session = sagemaker.Session()

s3 = boto3.resource('s3')
s3_client = boto3.client('s3')

BUCKET  = sagemaker_session.default_bucket()
print("BUCKET : ", BUCKET)

role = get_execution_role()
print("ROLE : ", role)

ACCOUNT_ID = boto3.client('sts').get_caller_identity()['Account']
REGION = boto3.Session().region_name

DATA_PREFIX = 'DEMO_MME_LINEAR_LEARNER'
HOUSING_MODEL_NAME = 'housing'
MULTI_MODEL_ARTIFACTS = 'multi_model_artifacts'

BUCKET :  sagemaker-us-east-1-622343165275
ROLE :  arn:aws:iam::622343165275:role/service-role/AmazonSageMaker-ExecutionRole-20220208T115633


In [33]:
#Create the SKLearn estimator with the sklearn_preprocessor.py as the script
from sagemaker.sklearn.estimator import SKLearn

script_path = 'scripts/sklearn_preprocessor_batch.py'

sklearn_estimator = SKLearn(
    entry_point=script_path,
    role=role,
    instance_type="ml.c4.xlarge",
    framework_version="0.20.0",
    sagemaker_session=sagemaker_session)

In [34]:
#Upload the raw training data to S3 bucket, to be accessed by SKLearn
train_inputs = []

for loc in LOCATIONS[:PARALLEL_TRAINING_JOBS]:

    train_input = sagemaker_session.upload_data(
        path='data/{}/train/train.csv'.format(loc),
        bucket=BUCKET,
        key_prefix='housing-data/{}/train'.format(loc)
    )
    
    train_inputs.append(train_input)
    print("Raw training data uploaded to : ", train_input)

Raw training data uploaded to :  s3://sagemaker-us-east-1-622343165275/housing-data/NewYork_NY/train/train.csv


In [35]:
##Launch multiple scikit learn training to process the raw synthetic data generated for multiple locations.
##Before executing this, take the training instance limits in your account and cost into consideration.

sklearn_estimators = []
sklearn_estimator_jobs = []

for index, loc in enumerate(LOCATIONS[:PARALLEL_TRAINING_JOBS]):
    print("sklearn_estimator fit input data at ", index , " for loc ", loc)
     
    job_name='scikit-learnestimator-{}'.format(strftime('%Y-%m-%d-%H-%M-%S', gmtime()))
    
    sklearn_estimator.fit({'train': train_inputs[index]}, job_name=job_name, wait=False)

    sklearn_estimators.append(sklearn_estimator)
    sklearn_estimator_jobs.append(job_name)
    
    time.sleep(1)

sklearn_estimator fit input data at  0  for loc  NewYork_NY


In [36]:
#Wait for the preprocessor jobs to finish
for job_name in sklearn_estimator_jobs:
    print('Waiting for job {} to complete...'.format(job_name))
    
    waiter = sm_client.get_waiter('training_job_completed_or_stopped')
    waiter.wait(TrainingJobName=job_name)

Waiting for job scikit-learnestimator-2022-08-04-00-09-56 to complete...


In [37]:
##Once the preprocessor is fit, use tranformer to preprocess the raw training data and store the transformed data right back into s3.
##Before executing this, take the training instance limits in your account and cost into consideration.

sklearn_estimator_transformers = []

for index, loc in enumerate(LOCATIONS[:PARALLEL_TRAINING_JOBS]):
    print("Transform the raw data at ", index , " for loc ", loc)
       
    sklearn_estimator = sklearn_estimators[index]
    
    transformer = sklearn_estimator.transformer(
        instance_count=1,
        instance_type='ml.m4.xlarge',
        assemble_with='Line',
        accept='text/csv'
    )
    
    sklearn_estimator_transformers.append(transformer)

Transform the raw data at  0  for loc  NewYork_NY


In [38]:
# Preprocess training input
preprocessed_train_data_path = []

for index, transformer in enumerate(sklearn_estimator_transformers):
    transformer.transform(train_inputs[index], content_type='text/csv', wait=False)
    print('STARTING: batch transform job: {}'.format(transformer.latest_transform_job.job_name))
    preprocessed_train_data_path.append(transformer.output_path)

STARTING: batch transform job: sagemaker-scikit-learn-2022-08-04-00-14-06-343


In [39]:
#Wait for all the batch transform jobs to finish
for transformer in sklearn_estimator_transformers: 
    job_name=transformer.latest_transform_job.job_name
    print('Waiting for TRANSFORM job {} to complete...'.format(job_name))
    
    waiter = sm_client.get_waiter('transform_job_completed_or_stopped')
    waiter.wait(TransformJobName=job_name)

Waiting for TRANSFORM job sagemaker-scikit-learn-2022-08-04-00-14-06-343 to complete...


In [40]:
##Download the preprocessed data, split into train and val, upload back to S3 in the same directory as tranformer output path
for index, transformer in enumerate(sklearn_estimator_transformers): 
    transformer_output_key='{}/{}'.format(transformer.latest_transform_job.job_name, 'train.csv.out') 
    
    preprocessed_data_download_dir = '{}/'.format("preprocessed-data/"+LOCATIONS[index])
    
    sagemaker_session.download_data(
        path=preprocessed_data_download_dir, 
        bucket=BUCKET,
        key_prefix=transformer_output_key
    )
    
    print('transformer_output_key: {}'.format(transformer_output_key ))
    print('Download directory: {}'.format(preprocessed_data_download_dir ))
    
    train_df = pd.read_csv('{}/{}'.format(preprocessed_data_download_dir,"train.csv.out"))
    
    #Spliting data into train and test in 70:30 ratio
    train, val = train_test_split(train_df, test_size=0.3)
    
    train.to_csv('{}{}'.format(preprocessed_data_download_dir,"train.csv"), sep=',', header=False, index=False)
    val.to_csv('{}{}'.format(preprocessed_data_download_dir,"val.csv"), sep=',', header=False, index=False)
    
    
    train_input = sagemaker_session.upload_data(
        path='{}/{}'.format(preprocessed_data_download_dir, 'train.csv'), 
        bucket=BUCKET,
        key_prefix='{}'.format(transformer.latest_transform_job.job_name, 'train.csv'))
    
    val_input = sagemaker_session.upload_data(
        path='{}/{}'.format(preprocessed_data_download_dir, 'val.csv'), 
        bucket=BUCKET,
        key_prefix='{}'.format(transformer.latest_transform_job.job_name, 'val.csv'))
    
    print(f"Transform:transformer_output_key={transformer_output_key}::Train:uploaded:{train_input}::Validation:{val_input}")   
    
    

transformer_output_key: sagemaker-scikit-learn-2022-08-04-00-14-06-343/train.csv.out
Download directory: preprocessed-data/NewYork_NY/
Transform:transformer_output_key=sagemaker-scikit-learn-2022-08-04-00-14-06-343/train.csv.out::Train:uploaded:s3://sagemaker-us-east-1-622343165275/sagemaker-scikit-learn-2022-08-04-00-14-06-343/train.csv::Validation:s3://sagemaker-us-east-1-622343165275/sagemaker-scikit-learn-2022-08-04-00-14-06-343/val.csv


In [41]:
##S3 location of the preprocessed data
for preprocessed_train_data in preprocessed_train_data_path: 
    print(preprocessed_train_data)

s3://sagemaker-us-east-1-622343165275/sagemaker-scikit-learn-2022-08-04-00-14-06-343


In [42]:
for index, loc in enumerate(LOCATIONS[:PARALLEL_TRAINING_JOBS]):
    preprocessed_data_download_dir = '{}/'.format("preprocessed-data/"+LOCATIONS[index])
    path='{}/{}'.format(preprocessed_data_download_dir, 'train.csv')

## Clean up<a id='CleanUp'></a>
Clean up the endpoint to avoid unneccessary costs.



In [ ]:
#Delete the endpoint and underlying model
predictor.delete_model() 
predictor.delete_endpoint()
for t in preprocessor_transformers:
    t.delete_model()